### 데이터 전처리


In [7]:
from util import preprocessing_simple_train
from util import month_to_day, make_mean
import pandas as pd
import numpy as np

mean_df = preprocessing_simple_train("fast")
tmp = month_to_day(mean_df['rainfall_train.ef_month']) + mean_df['rainfall_train.ef_day'] 

mean_df['daysin'] = np.sin(tmp)
mean_df['daycos'] = np.cos(tmp)
mean_df['day'] = tmp
mean_df = make_mean(mean_df,15)
r_df = pd.read_csv('./data/rainfall_train.csv')[['rainfall_train.stn4contest','rainfall_train.ef_year','rainfall_train.ef_month','rainfall_train.ef_day','rainfall_train.ef_hour','rainfall_train.class_interval']]
r_df = r_df[r_df['rainfall_train.class_interval'] != -999]

tmp = mean_df.groupby(by=['rainfall_train.ef_year','rainfall_train.stn4contest','day'])['v_median'].mean().reset_index().copy()
tmp = tmp.rename(columns={'v_median':'day_vmedian'})


mean_df = pd.merge(mean_df,tmp,how='left',on=['rainfall_train.ef_year','rainfall_train.stn4contest','day'])

_stn = [f"STN{format(i,'03')}" for i in range(1,21)]
mean_df['prev_vmedian'] = None
mean_df['post_vmedian'] = None


for i in mean_df['rainfall_train.ef_year'].unique(): 
    for j in _stn:
        tmp = mean_df[(mean_df['rainfall_train.stn4contest'] == j) & (mean_df['rainfall_train.ef_year'] == i)].sort_values(['day'])
        mean_df.loc[tmp.index,'prev_vmedian'] = (tmp['v_median'] - tmp['v_median'].shift(-5).fillna(tmp.loc[tmp.index[-1], 'v_median']))/5.0
        mean_df.loc[tmp.index,'post_vmedian'] = (tmp['v_median'].shift(5).fillna(tmp.loc[tmp.index[0], 'v_median']) - tmp['v_median'])/5.0



### 테스트/ 검증 데이터 결정
####  테스트데이터

In [13]:
ctrain_df.columns

Index(['rainfall_train.stn4contest', 'rainfall_train.dh_x',
       'rainfall_train.ef_year', 'rainfall_train.ef_month',
       'rainfall_train.ef_day', 'rainfall_train.ef_hour', 'rainfall_train.v01',
       'rainfall_train.v02', 'rainfall_train.v03', 'rainfall_train.v04',
       'rainfall_train.v05', 'rainfall_train.v06', 'rainfall_train.v07',
       'rainfall_train.v08', 'rainfall_train.v09', 'rainfall_train.vv',
       'rainfall_train.class_interval', 'rainfall_train.v00', 'v_max',
       'v_median', 'daysin', 'daycos', 'day', 'mean_vmax', 'mean_vmedian',
       'day_vmedian', 'prev_vmedian', 'post_vmedian', 'mean_class',
       'freq_class'],
      dtype='object')

In [17]:
from util import make_day2class, make_day2freqclass
from sklearn.preprocessing import StandardScaler
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from util import compute_csi

feature_list1 = ['rainfall_train.v01', 'rainfall_train.v02', 'rainfall_train.v03',
        'rainfall_train.v04', 'rainfall_train.v05', 'rainfall_train.v06',
        'rainfall_train.v07', 'rainfall_train.v08', 'rainfall_train.v09',
        'rainfall_train.v00', 'v_median', 'mean_vmedian', 'day_vmedian']

feature_list2 = ['rainfall_train.v01', 'rainfall_train.v02', 'rainfall_train.v03',
       'rainfall_train.v04', 'rainfall_train.v05', 'rainfall_train.v06',
       'rainfall_train.v07', 'rainfall_train.v08', 'rainfall_train.v09',
       'rainfall_train.v00', 'v_median', 'mean_vmedian', 'day_vmedian',
       'prev_vmedian', 'post_vmedian']

for _year in mean_df['rainfall_train.ef_year'].unique():
    train_df = mean_df[mean_df['rainfall_train.ef_year'] != _year].reset_index(drop=True)
    test_df = mean_df[mean_df['rainfall_train.ef_year'] == _year].reset_index(drop=True)
    rtrain_df = r_df[r_df['rainfall_train.ef_year'] != _year].reset_index(drop=True)
    rtest_df = r_df[r_df['rainfall_train.ef_year'] == _year].reset_index(drop=True)

    day2class = make_day2class(train_df)
    day2freqclass = make_day2freqclass(train_df)

    day2class=day2class.rename(columns={"rainfall_train.class_interval":"mean_class"})
    day2class['mean_class'] = StandardScaler().fit_transform(day2class[['mean_class']])

    day2freqclass=day2freqclass.rename(columns={"rainfall_train.class_interval":"freq_class"})
    train_df = pd.merge(train_df,day2class,how="left",on=['day']).copy()
    train_df = pd.merge(train_df,day2freqclass,how="left",on=['day']).copy()
    train_df['freq_class'] = train_df['freq_class'].fillna(0)
    test_df = pd.merge(test_df,day2class,how="left",on=['day']).copy()
    test_df = pd.merge(test_df,day2freqclass,how="left",on=['day']).copy()
    test_df['freq_class'] = test_df['freq_class'].fillna(0)
#### -----model hyperparameter -----###
    m_c = GradientBoostingClassifier(n_estimators=10,random_state=42,max_depth=8,criterion='squared_error')
#### -----model hyperparameter -----###

    ctrain_df = train_df[train_df['rainfall_train.class_interval'] != 0]
    ctest_df = test_df[test_df['rainfall_train.class_interval'] != 0]

    

    CX = ctrain_df[feature_list1]
    CX_V= ctest_df[feature_list1]
    CY = ctrain_df["rainfall_train.class_interval"]
    CY_V = ctest_df["rainfall_train.class_interval"]
    m_c.fit(CX,CY)
    CY_prev =m_c.predict(CX_V)

    print(f"{_year}년train: {compute_csi(CY,m_c.predict(CX))}")
    print(f"{_year}년valid: {compute_csi(CY_V,CY_prev)}")
    print()

A년train: 0.6469548667754215
A년valid: 0.21792656587473003

B년train: 0.6325075075075075
B년valid: 0.23435152729093642

C년train: 0.6222170686456401
C년valid: 0.21267102914931588



In [11]:
pd.Series(CY_prev).value_counts()

5    1546
2    1411
7     247
6     158
Name: count, dtype: int64